In [1]:
from sage.algebras.flag_algebras import *

In [2]:
from scipy.optimize import linprog
c = [-1, 4]
A = [[-3, 1], [1, 2]]
b = [6, 4]
x0_bounds = (None, None)
x1_bounds = (-3, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method="highs-ds")
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -22.0
              x: [ 1.000e+01 -3.000e+00]
            nit: 0
          lower:  residual: [       inf  0.000e+00]
                 marginals: [ 0.000e+00  6.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.900e+01  0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [29]:
#rounding based on ldl decomposition and continued fractions

#continued fraction rounding
def cfr(value, accuracy=1e3):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=accuracy:
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, accuracy=1e3):
    return [cfr(xx, accuracy) for xx in ls]

#rounding for matrices
def cfr_matrix(mat, accuracy=1e3):
    return [cfr_list(xx, accuracy) for xx in mat]

#rounding matrix based on LDL factoring
def cfr_ldl(mat, accuracy=1e3):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1], accuracy))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), accuracy))
    return P*L*D*L.T*P.T

#rounding matrix based on LDL factoring with given kernel
def cfr_ldl_corr(mat, kern, accuracy=1e3):
    # mat is a floating point symmetric positive semidefinite matrix
    # kern is a rational symmetric positive semidefinite matrix

    mat_ldl = matrix(mat).block_ldlt()
    kern_ldl = matrix(kern).block_ldlt()
    
    P_mat = matrix(QQ, mat_ldl[0])
    L_mat = P_mat * matrix(QQ, cfr_matrix(mat_ldl[1], accuracy)) * P_mat.T
    D_mat = P_mat * diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), accuracy)) * P_mat.T
    
    P_kern = kern_ldl[0]
    L_kern = P_kern * kern_ldl[1] * P_kern.T
    D_kern = P_kern * kern_ldl[2] * P_kern.T
    
    # Round L_mat and D_mat
    L_rounded = matrix(QQ, cfr_matrix(L_mat, accuracy))
    D_rounded = diagonal_matrix(QQ, cfr_list(D_mat.diagonal(), accuracy))
    
    # Get nonzero elements from kern and mat's L values
    kern_nonzero_L = [L_kern[ii] for ii in range(D_kern.nrows()) if D_kern[ii, ii] != 0]
    mat_nonzero_inds = [jj for jj in range(D_mat.nrows()) if D_mat[jj, jj] != 0]
    
    # Adjust L_mat based on kern's constraints
    for jj in mat_nonzero_inds:
        mjj = L_mat[jj]
        gmb = matrix(QQ, kern_nonzero_L + [mjj])
        Ggmb, Mgmb = gmb.gram_schmidt()
        L_mat[jj] = Ggmb[-1]
    
    # Reconstruct the matrix
    mat_rounded = L_mat * D_mat * L_mat.T
    
    return mat_rounded

In [77]:
def optimize_problem(self, target_element, target_size, maximize=True, \
                     ftypes=None, positives=None, \
                     certificate=False, rounding=None):
    r"""
    Try to maximize or minimize the value of `target_element`
    
    The algorithm calculates the multiplication tables and 
    sends the SDP problem to cvxopt.
    
    INPUT:

    - ``target_element`` -- Flag or FlagAlgebraElement; 
        the target whose density this function tries to
        maximize or minimize in large structures.
    - ``target_size`` -- integer; The program evaluates
        flags and the relations between them up to this
        size.
    - ``maximize`` -- boolean (default: `True`); 
    - ``ftypes`` -- list of ftypes (default: `None`); 
        the list of ftypes the program will consider,
        if not provided (or `None`) then generates all
        possible ftypes.
    - ``positives`` -- list of flag algebra elements, 
        optimizer will assume those are positive, can
        have different types.
    - ``certificate`` -- boolean (default: `False`);
        should it return a certificate of the bound.
    - ``rounding`` -- (default: `None`); if None, no 
        rounding, otherwise this specifies the accuracy

    OUTPUT: A bound for the optimization problem. If 
        certificate is requested then returns a density
        of the limit structure and the SOS proof matrices

    EXAMPLES::
    
    
    Mantel's theorem, calculate the maximum density
    of edges in triangle-free graphs ::

        sage: from sage.algebras.flag_algebras import *
        sage: GraphTheory.exclude(GraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]]))
        sage: x = GraphTheory.optimize_problem(GraphTheory(2, edges=[[0, 1]]), 3)
        ...
        sage: abs(x-0.5)<1e-6
        True
    
    Generalized Turan problem, for example maximum density of K_3
    in K_5 -free graphs. The complement is calculated, optimizing empty
    E_3 in E_5 -free graphs. They are equivalent

        sage: GraphTheory.exclude(GraphTheory(5))
        sage: x = GraphTheory.optimize_problem(GraphTheory(3), 5) # long time (5 second)
        ...
    
    
    Generalized Turan problem for hypergraphs, maximum density of K^3_4
    in K^3_5 -free 3-graphs. Again, the complement is calculated. This is 
    long and should not be normally tested

        sage: ThreeGraphTheory.exclude(ThreeGraphTheory(5))
        sage: ThreeGraphTheory.optimize_problem(ThreeGraphTheory(4), 6) # todo: not implemented
        ...
    
    
    The minimum number of transitive tournaments is attained at 
    a random tournament [CoRa2015]_::

        sage: x = TournamentTheory.optimize_problem( \
        ....: TournamentTheory(3, edges=[[0, 1], [0, 2], [1, 2]]), \
        ....: 3, maximize=False)
        ...
    
    
    Ramsey's theorem, the K_5 is the largest 2-colorable complete
    graph without monochromatic K_3. (see [LiPf2021]_) ::

        sage: RamseyGraphTheory.exclude(RamseyGraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]]))
        sage: x = RamseyGraphTheory.optimize_problem(RamseyGraphTheory(2), 4, maximize=False)
        ...
        sage: abs(x-0.2)<1e-6
        True
    
    .. NOTE::

        If `target_size` is too large, the calculation might take a 
        really long time. On a personal computer the following 
        maximum target_size values are recommended:
        -GraphTheory: 8
        -ThreeGraphTheory: 6
        -DiGraphTheory: 5
        -TournamentTheory: 9
        -PermutationTheory: 8
        -RamseyGraphTheory: 8
        -OVGraphTheory: 5
        -OEGraphTheory: 4
    """
    import time
    import sys

    try:
        from csdpy import solve_sdp
    except:
        print("No csdpy solver, can't run the optimizer!")
        return
    
    current = time.time()
    #calculate constraints from positive vectors
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    #calculate ftypes
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]

    print("Ftypes constructed in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])

    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)
    mat_inds = []
    mat_vals = []
    print("Block sizes done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    print("Block sizes are {}".format(block_sizes), flush=True)
    print("Calculating product matrices for {} ftypes and {} structures".format(len(ftypes), constraints), flush=True)
    
    pbar = None
    has_tqdm = True
    try:
        from tqdm import tqdm
        pbar = tqdm(enumerate(ftypes), file=sys.stdout)
    except:
        pbar = enumerate(ftypes)
        has_tqdm = False
    
    for ii, ftype in pbar:
        ns = (target_size + ftype.size())//2
        fls = self.generate_flags(ns, ftype)
        table = self.mul_project_table(ns, ns, ftype, [])
        for gg, mm in enumerate(table):
            dd = mm._dict()
            if len(dd)>0:
                inds, values = zip(*mm._dict().items())
                iinds, jinds = zip(*inds)
                for cc in range(len(iinds)):
                    if iinds[cc]>=jinds[cc]:
                        mat_inds.extend([gg+1, ii+1, iinds[cc]+1, 
                                         jinds[cc]+1])
                        mat_vals.append(values[cc])
        if has_tqdm: 
            pbar.set_description("{} is complete".format(ftype))
        else:
            print("{} is complete".format(ftype), flush=True)
    
    print("Table calculation done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    if maximize:
        avals = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        avals = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    for ii in range(len(constraints_vals)):
        mat_inds.extend([0, block_num, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    for gg in range(constraints):
        mat_inds.extend([gg+1, block_num-1, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(len(constraints_flags_vec)):
            mat_inds.extend([gg+1, block_num, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    print("Target and constraint calculation done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
    
    sdp_result = solve_sdp(block_sizes, list(avals), 
                           mat_inds, mat_vals)
    print("SDP finished in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()

    # check scipy is installed, so we can do LP for the rounding
    if rounding != None:
        try:
            from scipy.optimize import linprog
        except:
            print("SciPy is needed for the rounding!")
            rounding=None
    
    if rounding==None:
        #No rounding, just return the result
        if maximize:
            ret = max(-sdp_result['primal'], -sdp_result['dual'])
        else:
            ret = min(sdp_result['primal'], sdp_result['dual'])
        print("Result is {}".format(ret), flush=True)
        if certificate:
            ralg = FlagAlgebra(RR, self)
            vec = ralg(target_size, sdp_result['y'])
            ret = (ret, vec, sdp_result)
        return ret
    else:
        #
        # Rounding code
        #
        
        accuracy = 10**max(rounding, 2)
        
        #First get a good rounded y:
        print("Rounding dual", flush=True); current = time.time()
        yvec = vector(sdp_result['y'])
        onevec = one_vector.values()
        
        best_y = None
        best_err = 1000
        
        for ex in range(1, 8):
            ry = vector(cfr_list(yvec, 10**ex))
            prod = onevec*ry
            
            if prod != 0 and abs(prod-1)<best_err:
                best_err = abs(prod-1)
                best_y = ry/prod

        print("rounded y is ", best_y)
        
        #Then loop through the tables, keeping track the y-corrected and
        #normal slack values

        slack_base = vector(list(avals))
        slack_corr = vector(list(avals))
    
        print("initial slack is ", avals)
        
        correct_y = True
        
        pbar = None
        if has_tqdm and False:
            pbar = tqdm(enumerate(ftypes), file=sys.stdout)
        else:
            pbar = enumerate(ftypes)
        
        for ii, ftype in pbar:
            #calculation of the table entries
            ns = (target_size + ftype.size())//2
            fls = self.generate_flags(ns, ftype)
            table = self.mul_project_table(ns, ns, ftype, [])
            
            #if still correct, calculate the Z matrix
            if correct_y:
                Zii = None
                for jj, mat in enumerate(table):
                    if Zii==None:
                        Zii = mat*best_y[jj]
                    else:
                        Zii += mat*best_y[jj]
                if min(Zii.eigenvalues())<0:
                    correct_y = False

            #the two different X matrix rounding
            Xii_base = cfr_ldl(sdp_result['X'][ii], accuracy)
            Xii_base = vector(Xii_base.list())
            if correct_y:
                Xii_corr = cfr_ldl_corr(sdp_result['X'][ii], Zii, accuracy)
                Xii_corr = vector(Xii_corr.list())
            
            #update slacks
            for jj, mat in enumerate(table):
                #TODO: make this a sparse vector, as the table is sparse
                mvec = vector(mat.list())
                slack_base[jj] -= (Xii_base*mvec)
                if correct_y:
                    slack_corr[jj] -= Xii_corr*mvec
            
            if has_tqdm and False:
                pbar.set_description("{} is complete".format(ftype))
            else:
                print("{} is complete".format(ftype), flush=True)
        
        #
        # semidefinite effect on slacks is done
        # optimize the rest with LP
        #
        
        
        #
        # LP has form:
        #
        # linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, 
        #         bounds=(0, None), method='highs', callback=None, 
        #         options=None, x0=None, integrality=None)
        #
        # minimize_x c * x
        # s.t. A_ub * x <= b_ub
        #      A_eq * x == b_eq
        #      bound[0] <= x <= bounds[1]
        # 
        # want to use method='highs-ds' for simplex

        sdim = len(constraints_flags_vec)-2
        
        A_ub = matrix(QQ, [onevec] + constraints_flags_vec[:sdim])
        A_ub = -A_ub.T
        b_ub = list(-slack_corr) if correct_y else list(-slack_base)
        bounds = ([None] + [0]*sdim   ,   [None]*(1 + sdim))
        c = [1] + [0]*sdim
        
        print("DEBUG LOG: \n\n", A_ub, "\n\n", b_ub, "\n\n", bounds, "\n\n", c)
        
        res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs-ds")
        
        print("Dual rounding done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
        return res

In [78]:
G = GraphTheory
G.exclude(G(3))
optimize_problem(G, G(2), 3, certificate=True, rounding=3)

Ftypes constructed in 0.00s
Block sizes done in 0.01s
Block sizes are [2, -3, -2]
Calculating product matrices for 1 ftypes and 3 structures
Ftype on 1 points with edges=[] is complete: : 1it [00:00, 98.48it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.01s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -1.5751371e+01 Ad: 7.93e-01 Dobj: -1.8596991e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.3829292e+01 Ad: 9.44e-01 Dobj: -2.5639371e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -3.6548572e+00 Ad: 9.22e-01 Dobj: -2.8097530e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.7102979e-01 Ad: 8.42e-01 Dobj: -3.1080769e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.8576247e-01 Ad: 8.44e-01 Dobj: -4.8002387e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -5.0647721e-01 Ad: 8.85e-01 Dobj: -4.9296240e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -5.0060022e-01 Ad: 9.41e-01 Dobj: -4.9923793e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -5.0005080e-01 Ad: 1.00e+00 Dobj: -4.9996691e

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -0.16666666666666666
              x: [-1.667e-01]
            nit: 0
          lower:  residual: [       inf]
                 marginals: [ 0.000e+00]
          upper:  residual: [       inf]
                 marginals: [ 0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.333e-01  0.000e+00  3.333e-01]
                 marginals: [-0.000e+00 -1.000e+00 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [ ]:
def rounded_result(self, certificate, target_element, target_size, ftypes=None, maximize=True, positives=None):
    from tqdm import tqdm
    
    #
    # setup of problem values
    #
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    
    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)

    A_lin = matrix([(xx<<(target_size-xx.size())).values() for xx in constraints_flags])
    
    #
    # rounding target vector
    #
    
    ryv = vector(cfr_list(certificate[1].values()))
    if ryv*one_vector.values() != 1:
        
    
    
    if maximize:
        slacks = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        slacks = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    target = 0
    
    
    for ii in range(block_num):
        Xii = rXs[ii]
        if block_sizes[ii]!=-constraints and block_sizes<0:
            slacks -= Xii*A_lin
            target += vector(constraints_vals)*Xii

    print("target is ", target.n())

    for ff in tqdm(range(constraints)):
        for ii in range(block_num):
            if block_sizes[ii]<0:
                continue
            Xii = rXs[ii]
            slacks
    
    
    if maximize:
        target = -target
        slacks = -slacks
    return slacks, target

In [3]:
#Optimizing with flag as a target
#tries to maximize the quotient (flag.project() / ftype.project()) in target
G = GraphTheory
G.exclude(G(3))
resp = G.optimize_problem(G(2), 3, certificate=True)

Ftypes constructed in 0.00s
Block sizes done in 0.17s
Block sizes are [2, -3, -2]
Calculating product matrices for 1 ftypes and 3 structures
Ftype on 1 points with edges=[] is complete: : 1it [00:00, 318.55it/s]
Table calculation done in 0.01s
Target and constraint calculation done in 0.01s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -1.5751371e+01 Ad: 7.93e-01 Dobj: -1.8596991e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.3829292e+01 Ad: 9.44e-01 Dobj: -2.5639371e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -3.6548572e+00 Ad: 9.22e-01 Dobj: -2.8097530e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.7102979e-01 Ad: 8.42e-01 Dobj: -3.1080769e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.8576247e-01 Ad: 8.44e-01 Dobj: -4.8002387e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -5.0647721e-01 Ad: 8.85e-01 Dobj: -4.9296240e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -5.0060022e-01 Ad: 9.41e-01 Dobj: -4.9923793e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -5.0005080e-01 Ad: 1.00e+00 Dobj: -4.9996691

In [73]:
rXsp = certificate_rounding(resp[2]['X'])
y = resp[1]
ry = FlagAlgebra(QQ, G)(3, cfr_list(y.values()))

In [77]:
slacks, target = verify_rounded(G, rXsp, G(2), 3)

updated min slack is  -0.500000000000000
updated min slack is  -0.500000000000000
updated min slack is  6.94585762407944e-9
target is  -1919842718776245/3839685384212674


In [33]:
T = ThreeGraphTheory
T.exclude(T(5))
res = T.optimize_problem(T(4), 6, certificate=True)

Ftypes constructed in 0.00s
Block sizes done in 3.89s
Block sizes are [12, 63, 64, 64, 64, 64, -2102, -2]
Calculating product matrices for 6 ftypes and 2102 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]] is complete: : 6it [00:01,  4.23it/s]
Table calculation done in 1.42s
Target and constraint calculation done in 0.09s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.34e-01 Pobj: -6.0237881e+01 Ad: 2.53e-01 Dobj: -2.3386063e+01 
Iter:  2 Ap: 4.58e-01 Pobj: -1.0530716e+02 Ad: 7.34e-01 Dobj:  5.6351290e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.3486258e+02 Ad: 7.49e-01 Dobj: -1.1455141e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -1.3921113e+02 Ad: 9.14e-01 Dobj: -2.6443004e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.3857549e+02 Ad: 8.64e-01 Dobj: -1.5287208e-01 
Iter:  6 Ap: 5.47e-01 Pobj: -1.5652488e+02 Ad: 7.60e-01 Dobj: -1.0387829e-01 
Iter:  7 Ap: 7.96e-01 Pobj: -2.0045895e+02 Ad: 8.20e-01 Dobj: -7.2657418e-02 
Iter

In [ ]:
rXs = certificate_rounding(res[2]['X'])

In [78]:
slacks, target = verify_rounded(T, rXs, T(4), 6)

updated min slack is  -0.810042989905524
updated min slack is  -0.488407794495072
updated min slack is  -0.426426667272427
updated min slack is  -0.426426667272427
updated min slack is  -0.426426667272427
updated min slack is  -0.375000003387175
updated min slack is  -0.375000003387175
updated min slack is  -3.35224732045471e-9
target is  -5269447938477607/14051861167964838


In [59]:
def verify_rounded(self, rXs, target_element, target_size, ftypes=None, maximize=True, certificate=False, positives=None):
    from tqdm import tqdm
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    
    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)

    if maximize:
        slacks = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        slacks = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    target = 0
    A_lin = matrix([(xx<<(target_size-xx.size())).values() for xx in constraints_flags])
    
    for ii in range(block_num):
        Xii = rXs[ii]
        if block_sizes[ii]!=-constraints and block_sizes<0:
            slacks -= Xii*A_lin
            target += vector(constraints_vals)*Xii

    print("target is ", target.n())

    for ff in tqdm(range(constraints)):
        for ii in range(block_num):
            if block_sizes[ii]<0:
                continue
            Xii = rXs[ii]
            slacks
    
    
    if maximize:
        target = -target
        slacks = -slacks
    return slacks, target

IndentationError: expected an indented block after 'for' statement on line 48 (1869665241.py, line 52)

In [76]:
def verify_rounded(self, rXs, target_element, target_size, ftypes=None, maximize=True, certificate=False, positives=None):
    from tqdm import tqdm
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    
    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)

    if maximize:
        slacks = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        slacks = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    target = 0
    A_lin = matrix([(xx<<(target_size-xx.size())).values() for xx in constraints_flags])
    
    for ii in range(block_num):
        Xii = rXs[ii]
        if block_sizes[ii]>0:
            vxil = vector(Xii.list())
            ftype = ftypes[ii]
            ns = (target_size + ftype.size())//2
            fls = self.generate_flags(ns, ftype)
            table = self.mul_project_table(ns, ns, ftype, [])
            slacks -= vector([vector(table[xx].list())*vxil for xx in range(constraints)])
        elif block_sizes[ii]!=-constraints:
            slacks -= Xii*A_lin
            target += vector(constraints_vals)*Xii
        print("{}/{} block done, updated min slack is ".format(ii, block_num), min(slacks).n())

    print("target is ", target)
    
    if maximize:
        target = -target
        slacks = -slacks
    return slacks, target

In [16]:
# to check the block sizes
def get_block_sizes(self, target_size):
    theory = self
    flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in theory.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in flags]
    block_sizes = [len(theory.generate_flags((target_size + ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    return block_sizes, constraints